In [ ]:
import json

In [120]:
import cloudscraper
import geopandas as gpd
import pandas as pd
import h3

In [121]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [122]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [123]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

In [124]:
country_shape = states_df["geometry"].union_all()

In [125]:
h3_shape = h3.geo_to_h3shape(country_shape)
h3_cells = h3.h3shape_to_cells(h3_shape, res=5)

## Get McDonalds Data

In [110]:
scraper = cloudscraper.create_scraper()

In [111]:
restaurant_list = []
for h3_cell in tqdm(h3_cells, desc="Parsing H3 Cells"):
    lat, lng = h3.cell_to_latlng(h3_cell)
    url = (
        "https://www.mcdonalds.com/googleappsv2/geolocation?latitude="
        + str(lat)
        + "&longitude="
        + str(lng)
        + "&radius=100&maxResults=1000&country=us&language=en-us"
    )
    locations = scraper.get(url).text
    if not locations:
        continue
    features = json.loads(locations)["features"]
    for feature in features:
        state_code = feature["properties"]["subDivision"]
        restaurant_list.append(
            {"STATE": state_code, "geometry": Point(feature["geometry"]["coordinates"])}
        )

Parsing H3 Cells:   0%|          | 0/36446 [00:00<?, ?it/s]

In [112]:
mcdonalds_gdf = gpd.GeoDataFrame(restaurant_list, crs=4326)

In [113]:
mcdonalds_new_gdf = mcdonalds_gdf.drop_duplicates().reset_index()[["STATE", "geometry"]]

In [114]:
mcdonalds_new_gdf.to_file("data/mcdonalds_va_h3_5.gpkg")

## Get Population Data

In [126]:
state_populations = pd.read_excel(
    "data/NST-EST2023-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [127]:
state_populations_df = state_populations["NST-EST2023-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [128]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "POPULATION", "geometry", "NAME"]
]

In [129]:
states_with_population_df = states_with_population_df.dropna()

## Merge Data

In [136]:
state_counts = {}
for i in tqdm(range(len(mcdonalds_new_gdf)), desc="Parsing McDonalds"):
    try:
        state_code = states_df[
            states_df.intersects(mcdonalds_new_gdf.iloc[i]["geometry"])
        ]["STUSPS"].iloc[0]
    except IndexError:
        print(i)
        continue

    if state_code not in state_counts:
        state_counts[state_code] = 1
    else:
        state_counts[state_code] += 1

Parsing McDonalds:   0%|          | 0/13512 [00:00<?, ?it/s]

8825


In [139]:
mcdonalds_per_state_df = pd.Series(state_counts, name="stores")
mcdonalds_per_state_df = mcdonalds_per_state_df.reset_index()
mcdonalds_per_state_df = mcdonalds_per_state_df.rename(columns={"index": "STUSPS"})

In [141]:
mcdonalds_per_state_df = states_with_population_df.merge(
    mcdonalds_per_state_df, on="STUSPS", how="left"
)
mcdonalds_per_state_df = mcdonalds_per_state_df.fillna(0)

In [142]:
mcdonalds_per_state_df["per_1000"] = mcdonalds_per_state_df["stores"] / (
    mcdonalds_per_state_df["POPULATION"] / 1000
)
mcdonalds_per_state_df["per_10k"] = mcdonalds_per_state_df["stores"] / (
    mcdonalds_per_state_df["POPULATION"] / 10_000
)
mcdonalds_per_state_df["per_100k"] = mcdonalds_per_state_df["stores"] / (
    mcdonalds_per_state_df["POPULATION"] / 100000
)
mcdonalds_per_state_df["per_500k"] = mcdonalds_per_state_df["stores"] / (
    mcdonalds_per_state_df["POPULATION"] / 500_000
)
mcdonalds_per_state_df["per_1m"] = mcdonalds_per_state_df["stores"] / (
    mcdonalds_per_state_df["POPULATION"] / 1_000_000
)
mcdonalds_per_state_df["per_capita"] = (
    mcdonalds_per_state_df["POPULATION"] / mcdonalds_per_state_df["stores"]
)

In [146]:
mcdonalds_per_state_df = mcdonalds_per_state_df.to_crs(9311)

In [147]:
mcdonalds_per_state_df.to_file("data/mcdonalds_per_state.gpkg")